In [1]:
import os
os.environ["THEANO_FLAGS"] = 'cuda.root=/usr/local/cuda,floatX=float32,device=gpu1,force_device=False,lib.cnmem=.75'

import theano
print(theano.config.device)

import mhcflurry, seaborn, numpy, pandas, pickle, sklearn, collections, scipy, time
import mhcflurry.dataset
import fancyimpute, locale

import sklearn.metrics
import sklearn.cross_validation

def print_full(x):
    pandas.set_option('display.max_rows', len(x))
    print(x)
    pandas.reset_option('display.max_rows')
    
%matplotlib inline


ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.
ERROR:theano.sandbox.cuda:nvcc compiler not found on $PATH. Check your nvcc installation and try again.


Couldn't import dot_parser, loading of dot files will not be possible.
gpu1


Using Theano backend.


In [2]:
max_ic50 = 50000
min_peptides_to_consider_allele = 10
data_dir = "../data/"

In [3]:
all_train_data = mhcflurry.dataset.Dataset.from_csv(data_dir + "bdata.2009.mhci.public.1.txt")

In [5]:
imputed_train_data = all_train_data.impute_missing_values(
    fancyimpute.MICE(n_imputations=50, n_burn_in=5),
    min_observations_per_peptide=5,
    min_observations_per_allele=2,
)


Dropping 19832 peptides with <5 observations
Dropping 18 alleles with <2 observations: ['ELA-A1', 'Gogo-B0101', 'HLA-A0302', 'HLA-A26', 'HLA-A6601', 'HLA-B1402', 'HLA-B2701', 'HLA-B2702', 'HLA-B3508', 'HLA-B4201', 'HLA-B44', 'HLA-E0101', 'Mamu-A07', 'Mamu-B04', 'Mamu-B52', 'Patr-A0602', 'Patr-B0901', 'Patr-B1701']
[MICE] Completing matrix with shape (11707, 88)
[MICE] Starting imputation round 1/55, elapsed time 0.036
[MICE] Starting imputation round 2/55, elapsed time 2.495
[MICE] Starting imputation round 3/55, elapsed time 5.252
[MICE] Starting imputation round 4/55, elapsed time 8.014
[MICE] Starting imputation round 5/55, elapsed time 10.461
[MICE] Starting imputation round 6/55, elapsed time 12.813
[MICE] Starting imputation round 7/55, elapsed time 15.337
[MICE] Starting imputation round 8/55, elapsed time 17.909
[MICE] Starting imputation round 9/55, elapsed time 20.483
[MICE] Starting imputation round 10/55, elapsed time 23.120
[MICE] Starting imputation round 11/55, elapsed t

In [7]:
imputed_train_data.to_dataframe()

allele         peptide       affinity  \
allele   peptide                                                   
H-2-KD   NTYLFNILYK        H-2-KD      NTYLFNILYK    6734.961206   
         NLSWLSLDV         H-2-KD       NLSWLSLDV   12053.730611   
         ETDVMTRGQ         H-2-KD       ETDVMTRGQ   10121.158896   
         SMMVILPDK         H-2-KD       SMMVILPDK    2628.947930   
         FVNRYGVAY         H-2-KD       FVNRYGVAY     468.816409   
         LQMAGVEVR         H-2-KD       LQMAGVEVR    4578.273866   
         YLKNYKNFDY        H-2-KD      YLKNYKNFDY   32214.714425   
         YLPEVISTIA        H-2-KD      YLPEVISTIA   21037.856626   
         QQLRVESSSK        H-2-KD      QQLRVESSSK   45472.081391   
         AQNISFKSI         H-2-KD       AQNISFKSI    1106.884704   
         KLFTIAMWL         H-2-KD       KLFTIAMWL    9488.556542   
         EVGTNFGTI         H-2-KD       EVGTNFGTI    2932.611165   
         EVIPYTPAM         H-2-KD       EVIPYTPAM   28111.453423   
         FPHCLAFSY         H-2-KD       FPHCLAFSY   25347.928386   
         WMMWYWGPSL        H-2-KD      WMMWYWGPSL     817.868620   
         SLVITYCLV         H-2-KD       SLVITYCLV     855.143061   
         AIAQSSMTK         H-2-KD       AIAQSSMTK     290.670836   
         ETRSFTTHF         H-2-KD       ETRSFTTHF  347757.036990   
         KLFPRLPGI         H-2-KD       KLFPRLPGI      70.605931   
         HPRHYATIM         H-2-KD       HPRHYATIM   16309.975614   
         GLLEWIFRA         H-2-KD       GLLEWIFRA  113846.730117   
         FASFYYIWK         H-2-KD       FASFYYIWK    2621.859787   
         LAKAIITPI         H-2-KD       LAKAIITPI   55863.687808   
         AILSLNLRI         H-2-KD       AILSLNLRI    1201.411631   
         IFREIASSMK        H-2-KD      IFREIASSMK   20061.133809   
         KLRNWQWWRL        H-2-KD      KLRNWQWWRL   85692.448912   
         NPAACSYMV         H-2-KD       NPAACSYMV    4200.128849   
         KPKVASEAF         H-2-KD       KPKVASEAF   17385.483647   
         NYMPYVFTLL        H-2-KD      NYMPYVFTLL    8685.966669   
         QTSYQYLII         H-2-KD       QTSYQYLII    1587.945548   
...                           ...             ...            ...   
Mamu-A01 AVDADDSHF       Mamu-A01       AVDADDSHF      16.909167   
         AINNRICVM       Mamu-A01       AINNRICVM      55.243843   
         QFNQMMNPSH      Mamu-A01      QFNQMMNPSH     458.788793   
         NALEKALRW       Mamu-A01       NALEKALRW      37.070190   
         LVSDYCNVLNKEFT  Mamu-A01  LVSDYCNVLNKEFT     264.451089   
         DQTHIKTIA       Mamu-A01       DQTHIKTIA     205.406095   
         SCSFKVGHH       Mamu-A01       SCSFKVGHH     133.952348   
         FTDNNELEF       Mamu-A01       FTDNNELEF      50.596121   
         LRSEAFEYY       Mamu-A01       LRSEAFEYY     381.004687   
         VPSHISSLI       Mamu-A01       VPSHISSLI      64.516015   
         RLAKLTEAI       Mamu-A01       RLAKLTEAI      48.270046   
         DPRRLVQLL       Mamu-A01       DPRRLVQLL      23.804389   
         IILAALFMY       Mamu-A01       IILAALFMY      12.917571   
         SIGNVTRLFY      Mamu-A01      SIGNVTRLFY     236.233321   
         DVLPFDIKY       Mamu-A01       DVLPFDIKY     348.713861   
         AMITYITRK       Mamu-A01       AMITYITRK     220.409933   
         EQRLPLLPK       Mamu-A01       EQRLPLLPK     352.846162   
         AEMKTDAATLA     Mamu-A01     AEMKTDAATLA     781.337129   
         ITPNNLNKI       Mamu-A01       ITPNNLNKI       1.000000   
         FQPQNGQFI       Mamu-A01       FQPQNGQFI    2272.727273   
         ETSFIRNCA       Mamu-A01       ETSFIRNCA     214.916427   
         SEAREHLKNG      Mamu-A01      SEAREHLKNG      23.037454   
         RGPGRAFVTI      Mamu-A01      RGPGRAFVTI     284.003624   
         NLRCHSAHV       Mamu-A01       NLRCHSAHV      19.737888   
         CINGACWTV       Mamu-A01       CINGACWTV       7.361306   
         FPRCRYVHK       Mamu-A01       FPRCRYVHK      59.201928   
         QSFLFW

In [8]:
validation_df = pandas.read_csv("../data/combined_test_BLIND_dataset_from_kim2013.csv")
validation_df

,allele,peptide,length,meas,netmhc,netmhcpan,smmpmbec_cpp
0,H-2-DB,AAACNVATA,9,657.657837,154.881662,711.213514,438.530698
1,H-2-DB,AAFEFVYV,8,30831.879502,6456.542290,785.235635,10351.421667
2,H-2-DB,AAFVNDYSL,9,77.446180,17.458222,7.516229,28.054336
3,H-2-DB,AAIANQAAV,9,1.999862,9.638290,9.749896,25.703958
4,H-2-DB,AAIANQAVV,9,1.517050,8.550667,8.336812,28.773984
5,H-2-DB,AAIENYVRF,9,37.844258,252.348077,114.815362,187.068214
6,H-2-DB,AAINFITTM,9,3.155005,199.986187,389.045145,200.909281
7,H-2-DB,AAIPAPPPI,9,3243.396173,1059.253725,493.173804,295.120923
8,H-2-DB,AAKLNRPPL,9,654.636174,66.374307,77.268059,38.459178
9,H-2-DB,AALDMVDAL,9,229.614865,547.015963,597.035287,225.423921


In [16]:
validation_allele_counts = validation_df.allele.value_counts()
validation_allele_counts_dict = dict(validation_allele_counts)
train_allele_counts = all_train_data._df.allele.value_counts()
train_allele_counts_dict = dict(train_allele_counts)
for common_allele in sorted(
        set(validation_allele_counts.keys()).intersection(set(train_allele_counts.keys())),
        key=lambda x: train_allele_counts[x]):
    print("\t%s %d %d" % (
            common_allele,
            train_allele_counts_dict[common_allele],
            validation_allele_counts_dict[common_allele]))

	HLA-B3801 136 351
	HLA-A2602 202 413
	HLA-A2603 205 312
	HLA-B0803 217 234
	HLA-B1509 346 466
	HLA-B1503 429 165
	HLA-B2703 433 441
	H-2-KD 452 229
	HLA-B0802 487 509
	HLA-A2501 519 416
	HLA-A3201 640 449
	HLA-A8001 782 379
	HLA-B1517 846 582
	HLA-B4002 866 74
	HLA-B3901 886 641
	HLA-B4501 889 65
	HLA-B4403 913 378
	Mamu-A02 938 388
	HLA-B5301 1018 485
	HLA-B5401 1019 79
	HLA-B4601 1424 378
	HLA-A3002 1430 420
	HLA-B4402 1705 411
	HLA-B5101 1734 854
	HLA-B5701 1857 815
	HLA-A2301 2021 391
	HLA-A3001 2040 660
	HLA-B1801 2052 503
	HLA-A6901 2079 470
	Mamu-A01 2264 274
	HLA-B0801 2267 940
	HLA-A2902 2397 118
	HLA-B3501 2397 542
	HLA-A2402 2533 573
	HLA-B5801 2564 445
	HLA-B4001 2718 407
	HLA-A2601 2894 1333
	HLA-B2705 3028 314
	HLA-A3301 3040 473
	HLA-A6801 3184 527
	HLA-B1501 3213 633
	H-2-DB 3216 564
	H-2-KB 3407 558
	HLA-B0702 3412 813
	HLA-A0101 3725 696
	HLA-A0202 3919 126
	HLA-A6802 4768 669
	HLA-A3101 4796 724
	HLA-A0206 4827 682
	HLA-A1101 5399 723
	HLA-A0203 5542 651
	HLA-A0301 

In [20]:
alleles = sorted(train_allele_counts.index[
    (train_allele_counts >= min_peptides_to_consider_allele)
    & (train_allele_counts.index.isin(validation_allele_counts.index))
], key=lambda allele: -1 * train_allele_counts[allele])
alleles
not_dropped = []
for allele in alleles:
    sub = all_train_data.get_allele(allele)._df
    if (sub.affinity < 500).sum() < 5 or (sub.affinity > 500).sum() < 5:
        print("Dropping allele %s" % allele)
    else:
        not_dropped.append(allele)
alleles = not_dropped
len(alleles)

Dropping allele HLA-B2703
Dropping allele HLA-B3801


51

In [29]:
dropout_probabilities = [0.5]
embedding_output_dims_and_layer_sizes_list = [(32, [64])] # , (8, [4])]
activations = ["tanh"]

models_params_list = []

for model_num in range(10):
    for negative_samples in [100]:
        for impute in [False, True]:
            for dropout_probability in dropout_probabilities:
                for (embedding_output_dim, layer_sizes) in embedding_output_dims_and_layer_sizes_list:
                    for activation in activations:
                        models_params_list.append(dict(
                            negative_samples=negative_samples,
                            impute=impute,
                            dropout_probability=dropout_probability,  
                            embedding_output_dim=embedding_output_dim,
                            layer_sizes=layer_sizes,
                            activation=activation))

print("%d models" % len(models_params_list))
models_params_explored = set.union(*[set(x) for x in models_params_list])
models_params_explored


20 models


{'activation',
 'dropout_probability',
 'embedding_output_dim',
 'impute',
 'layer_sizes',
 'negative_samples'}

In [30]:
def make_scores(ic50_y, ic50_y_pred, sample_weight=None, threshold_nm=500):     
    y_pred = mhcflurry.regression_target.ic50_to_regression_target(ic50_y_pred, max_ic50)
    try:
        auc = sklearn.metrics.roc_auc_score(ic50_y <= threshold_nm, y_pred, sample_weight=sample_weight)
    except ValueError:
        auc = numpy.nan
    try:
        f1 = sklearn.metrics.f1_score(ic50_y <= threshold_nm, ic50_y_pred <= threshold_nm, sample_weight=sample_weight)
    except ValueError:
        f1 = numpy.nan
    try:
        tau = scipy.stats.kendalltau(ic50_y_pred, ic50_y)[0]
    except ValueError:
        tau = numpy.nan
    
    return dict(
        auc=auc,
        f1=f1,
        tau=tau,
    )    


In [31]:
models_and_scores = {}
validation_df_with_mhcflurry = validation_df.copy()


In [ ]:
# train and test models, adding columns to validation_df_with_mhcflurry
pandas.DataFrame(models_params_list).to_csv("../data/validation_models.csv", index=False)

def make_and_fit_model(allele, original_params):
    params = dict(original_params)
    impute = params.pop("impute")
    
    negative_samples = params.pop("negative_samples")
    
    model = mhcflurry.Class1BindingPredictor.from_hyperparameters(max_ic50=max_ic50, **params)
    print("Fitting model for allele %s (%d + %d): %s" % (
            allele,
            len(all_train_data.groupby_allele_dictionary()[allele]),
            len(imputed_train_data.groupby_allele_dictionary()[allele]),
            str(original_params)))
    t = -time.time()
    model.fit_dataset(
        all_train_data.get_allele(allele),
        pretraining_dataset=imputed_train_data.get_allele(allele) if impute else None,
        verbose=False,
        batch_size=128,
        n_training_epochs=250,
        n_random_negative_samples=negative_samples)
    t += time.time()
    print("Trained in %d sec" % t)
    return model

for (i, allele) in enumerate(alleles):
    if allele not in validation_df_with_mhcflurry.allele.unique():
        print("Skipping allele %s: not in test set" % allele)
        continue
    if allele in models_and_scores:
        print("Skipping allele %s: already done" % allele)
        continue
    values_for_allele = []
    for (j, params) in enumerate(models_params_list):
        print("Allele %d model %d" % (i, j))
        model = make_and_fit_model(allele, params)
        predictions = model.predict(
            list(validation_df_with_mhcflurry.ix[validation_df_with_mhcflurry.allele == allele].peptide))
        print("test set size: %d" % len(predictions))
        validation_df_with_mhcflurry.loc[(validation_df_with_mhcflurry.allele == allele),
                                         ("mhcflurry %d" % j)] = predictions
        scores = make_scores(validation_df_with_mhcflurry.ix[validation_df.allele == allele].meas,
                            predictions)
        print(scores)
        values_for_allele.append((params, scores))
        
    models_and_scores[allele] = values_for_allele
    
    # Write out all data after each allele.
    validation_df_with_mhcflurry_results = validation_df_with_mhcflurry.ix[validation_df_with_mhcflurry.allele.isin(models_and_scores)]
    validation_df_with_mhcflurry_results.to_csv("../data/validation_predictions_full.csv", index=False)
    
    scores_df = collections.defaultdict(list)
    predictors = validation_df_with_mhcflurry_results.columns[4:]

    for (allele, grouped) in validation_df_with_mhcflurry_results.groupby("allele"):
        scores_df["allele"].append(allele)
        scores_df["test_size"].append(len(grouped.meas))
        for predictor in predictors:
            scores = make_scores(grouped.meas, grouped[predictor])
            for (key, value) in scores.items():
                scores_df["%s_%s" % (predictor, key)].append(value)

    scores_df = pandas.DataFrame(scores_df)
    scores_df["train_size"] = [
        len(all_train_data.groupby_allele_dictionary()[a])
        for a in scores_df.allele
    ]

    scores_df.index = scores_df.allele
    scores_df.to_csv("../data/validation_scores.csv", index=False)
        


Allele 0 model 0
Fitting model for allele HLA-A0201 (9565 + 11707): {'impute': False, 'layer_sizes': [64], 'dropout_probability': 0.5, 'activation': 'tanh', 'embedding_output_dim': 32, 'negative_samples': 100}

In [ ]:
print_full(scores_df[["train_size", "test_size"]].sort("train_size", inplace=False))

In [ ]:
2